#### Supervised by Mentor Videsh Jagai and implemented with Dr.Shannon Walker

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd    # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd 
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import shap as shap


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_columns', None)  

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv
/kaggle/input/widsdatathon2024-challenge1/training.csv
/kaggle/input/widsdatathon2024-challenge1/test.csv


In [2]:
BreastCancer_Train_Data = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
BreastCancer_Test_Data = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/test.csv')
Sample_Sub=pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv')

In [3]:
BreastCancer_Train_Data.drop(columns=['patient_id'],inplace=True)
BreastCancer_Test_Data.drop(columns=['patient_id'],inplace=True)

In [4]:
BreastCancer_Test_Data['DiagPeriodL90D'] = 2
df_FulDat = pd.concat([BreastCancer_Train_Data,BreastCancer_Test_Data])

In [5]:
#checking that test and train has the same shape
print(f"Train data: {BreastCancer_Train_Data.shape} \nTest data: {BreastCancer_Test_Data.shape}")

Train data: (12906, 82) 
Test data: (5792, 82)


In [6]:
Tr_Numerical_ColFea = BreastCancer_Train_Data.select_dtypes(exclude=['object']).columns
Tr_Categorical_ColFea = BreastCancer_Train_Data.select_dtypes(include=['object']).columns

In [7]:
# Investigation of columns with missing values
Missing_Col = [Col for Col in BreastCancer_Train_Data.columns if BreastCancer_Train_Data[Col].isnull().any()]

# Determining by counting the number of missing values per column
MissVal_C = BreastCancer_Train_Data.isna().sum()

In [8]:

# Generation of a DataFrame with columns and their assiociating  missing values count 
MissingVal_DF= pd.DataFrame(MissVal_C.rename('Total_Missing_Values_')).loc[MissVal_C.ne(0)]


# Generation of a function getting the intersection of two lists
def Inters(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [9]:
from sklearn.impute import SimpleImputer
# Find missing columns that are categorical or numerical
Categ_M = Inters(Missing_Col, Tr_Categorical_ColFea)
Numer_M = Inters(Missing_Col, Tr_Numerical_ColFea)

# Imputation of following missing values in categorical features
if len(Categ_M) > 0:
    Categ_Imouter = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")
    BreastCancer_Train_Data[Categ_M] = Categ_Imouter.fit_transform(BreastCancer_Train_Data[Categ_M])
    BreastCancer_Test_Data[Categ_M] = Categ_Imouter.fit_transform(BreastCancer_Test_Data[Categ_M])



In [10]:
# Use Simple Imputer for numerical features (using median as an example)
Numer_imputer = SimpleImputer(missing_values=np.nan, strategy="median")
BreastCancer_Train_Data[Numer_M] = Numer_imputer.fit_transform(BreastCancer_Train_Data[Numer_M])
BreastCancer_Test_Data[Numer_M] = Numer_imputer.fit_transform(BreastCancer_Test_Data[Numer_M])

In [11]:
# Check for missing values after imputation
print(BreastCancer_Train_Data.isna().sum().T)
print(BreastCancer_Test_Data.isna().sum().T)

patient_race      0
payer_type        0
patient_state     0
patient_zip3      0
patient_age       0
                 ..
veteran           0
Ozone             0
PM25              0
N02               0
DiagPeriodL90D    0
Length: 82, dtype: int64
patient_race      0
payer_type        0
patient_state     0
patient_zip3      0
patient_age       0
                 ..
veteran           0
Ozone             0
PM25              0
N02               0
DiagPeriodL90D    0
Length: 82, dtype: int64


In [13]:
df_FulDat = pd.concat([BreastCancer_Train_Data,BreastCancer_Test_Data])

In [14]:
df_FulDat

,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,metastatic_first_novel_treatment,metastatic_first_novel_treatment_type,Region,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,unknown,MEDICAID,CA,924,84,F,28.19,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,unknown,unknown,West,Pacific,31437.750000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,11.557143,7.571429,4.000000,2.100000,49.857143,50.142857,36.571429,11.885714,47.114286,4.442857,3.928571,52.228571,52996.28571,3.142857,4.000000,6.157143,5.142857,6.271429,10.142857,13.300000,20.000000,12.742857,11.571429,7.528571,19.100000,24563.57143,44.585714,8674.500000,2.646343e+05,1165.000000,37.442857,33.257143,29.200000,25.914286,8.357143,3.257143,11.614286,39.557143,61.528571,8.471429,13.428571,0.000000,44.100000,13.100000,5.100000,1.485714,0.342857,27.114286,8.757143,66.685714,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,unknown,unknown,West,Pacific,39121.878790,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,14.060606,10.248485,5.951515,3.503030,49.893939,50.106061,50.245455,9.827273,35.290909,4.651515,3.622727,61.736364,102741.63640,2.327273,1.536364,2.648485,2.178788,2.409091,5.163636,7.972727,13.936364,12.469697,19.760606,29.596970,49.357576,41287.27273,61.463636,11725.666670,6.776885e+05,2003.125000,34.753125,14.230303,19.987879,29.796970,23.739394,12.245455,35.984848,47.918182,65.230303,5.103030,15.224242,0.027273,54.030303,2.527273,20.827273,0.587879,0.300000,11.645455,10.081818,37.948485,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,C773,unknown,unknown,South,West South Central,21996.683330,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,12.770000,11.426667,6.565000,2.811667,50.123333,49.876667,55.753333,12.330000,27.195000,4.710000,3.260667,55.801667,85984.74138,2.483333,1.305000,2.716667,2.938333,2.766667,6.763333,12.061667,15.835000,13.560000,20.875000,18.680000,39.555000,40399.03333,72.745000,7786.583333,2.377131e+05,1235.907407,29.358491,10.811667,27.038333,32.368333,19.678333,10.115000,29.793333,37.308475,66.428333,4.560000,13.722034,3.650847,75.820000,9.231667,3.618333,0.463333,0.146667,3.816667,6.898333,19.370000,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,White,COMMERCIAL,CA,926,45,F,28.19,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,unknown,unknown,West,Pacific,32795.325580,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,14.847619,12.280952,8.216667,4.759524,49.066667,50.933333,52.604762,11.623810,31.142857

In [15]:
df_FulDat.isna().sum()

patient_race      0
payer_type        0
patient_state     0
patient_zip3      0
patient_age       0
                 ..
veteran           0
Ozone             0
PM25              0
N02               0
DiagPeriodL90D    0
Length: 82, dtype: int64

In [16]:
from sklearn.preprocessing import OrdinalEncoder

# Initialize the encoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# Loop through each categorical column
for col in Tr_Categorical_ColFea:
    # Fit the encoder on the training data
    encoder.fit(df_FulDat[[col]])
    
    # Transform both training and test data
    df_FulDat[col] = encoder.transform(df_FulDat[[col]])
    

Categ_Cols = ['breast_cancer_diagnosis_code','metastatic_cancer_diagnosis_code','patient_zip3','patient_age','payer_type',
        'patient_state','breast_cancer_diagnosis_desc']


In [17]:
BreastCancer_Train_Data = df_FulDat[df_FulDat['DiagPeriodL90D']!=2]
BreastCancer_Test_Data = df_FulDat[df_FulDat['DiagPeriodL90D']==2].drop(columns=['DiagPeriodL90D'])

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import numpy as np

X = BreastCancer_Train_Data[Categ_Cols+['DiagPeriodL90D']].drop(columns=['DiagPeriodL90D'], axis=1)

y = BreastCancer_Train_Data['DiagPeriodL90D']


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Inizialize booster and hyperparameters{ depth, l2_leaf_reg, learning_rate, colsample_bylevel,
## subsample, random_seed, eval_metric, verbose, loss_function}

params = {
'depth':2,
 'l2_leaf_reg': 0.0687356140243106,
 'learning_rate':0.0788501063398756,
'colsample_bylevel': 0.7017148360901887,
 'subsample': 0.814719415526711,
    'random_seed': 42,
    'eval_metric': 'AUC',
    'verbose': False,
    'loss_function': 'Logloss',
}


AUC_Scores = []
Test_Pred = []

for Train_IDX, Test_IDX in cv.split(X, y):
    X_train, X_test = X.iloc[Train_IDX], X.iloc[Test_IDX]
    y_train, y_test = y.iloc[Train_IDX], y.iloc[Test_IDX]
    
    ## CatBoostClassifier class with array-like data with numerical, categorical 
    ## and  by extension also with embedding features. 
    
    ## Purpose : i) Training and applying models for the classification problems. Provides
    ## compatibility with the scikit-learn tools.The default optimized objective depends 
    ## on various conditions:
    
    ## Logloss — The target has only two different values or the target_border parameter is not None.
    ## MultiClass — The target has more than two different values and the border_count parameter is None.
    model_catboost = CatBoostClassifier(**params)
    
   
    model_catboost.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)
    
    Pred_ = model_catboost.predict_proba(X_test)[:, 1]
    PredS_T = model_catboost.predict_proba(BreastCancer_Test_Data[Categ_Cols])[:, 1]
    Test_Pred.append(PredS_T)
    
    AUC_SinSc = roc_auc_score(y_test, Pred_)
    AUC_Scores.append(AUC_SinSc)
    print(f"AUC Score: {AUC_SinSc}")
    


AUC Score: 0.8151655751963368
AUC Score: 0.8087268602099321
AUC Score: 0.7997305426023094
AUC Score: 0.806365506892322
AUC Score: 0.8078222669504004


In [21]:
# Ortalama AUC skorunu yazdır
print(f"Ortalama AUC Skoru: {np.mean(AUC_Scores)}")
print(pd.DataFrame([1 if prob >= 0.5 else 0 for prob in np.mean(Test_Pred,axis=0)], columns=['Test_Pred'])['Test_Pred'].value_counts())

Ortalama AUC Skoru: 0.8076055031336168
Test_Pred
1    4405
0    1387
Name: count, dtype: int64


In [22]:

Sample_Sub['DiagPeriodL90D'] = np.mean(Test_Pred,axis=0)


In [23]:
Sample_Sub.head()

,patient_id,DiagPeriodL90D
0,573710,0.796870
1,593679,0.778969
2,184532,0.787458
3,447383,0.818834
4,687972,0.818649


In [24]:
Sample_Sub.to_csv('submission.csv',index=False)